# Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

### Context
Previously we saw that the model told us how to to change the tire, however we had to manually provide it with the relevant data and provide the contex ourselves. We explored the approach to leverage the model availabe under Bedrock and ask questions based on it's knowledge learned during training as well as providing manual context. While that approach works with short documents or single-ton applications, it fails to scale to enterprise level question answering where there could be large enterprise documents which cannot all be fit into the prompt sent to the model. 

### Pattern
We can improve upon this process by implementing an architecure called Retreival Augmented Generation (RAG). RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. 

In this notebook we explain how to approach the pattern of Question Answering to find and leverage the documents to provide answers to the user questions.

### Challenges
- How to manage large document(s) that exceed the token limit
- How to find the document(s) relevant to the question being asked

### Proposal
To the above challenges, this notebook proposes the following strategy
#### Prepare documents
![Embeddings](./images/Embeddings_lang.png)

Before being able to answer the questions, the documents must be processed and a stored in a document store index
- Load the documents
- Process and split them into smaller chunks
- Create a numerical vector representation of each chunk using Amazon Bedrock Titan Embeddings model
- Create an index using the chunks and the corresponding embeddings
#### Ask question
![Question](./images/Chatbot_lang.png)

When the documents index is prepared, you are ready to ask the questions and relevant documents will be fetched based on the question being asked. Following steps will be executed.
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks
- Add those chunks as part of the context in the prompt
- Send the prompt to the model under Amazon Bedrock
- Get the contextual answer based on the documents retrieved

## Usecase
#### Dataset
To explain this architecture pattern we are using the documents from IRS. These documents explain topics such as:
- Original Issue Discount (OID) Instruments
- Reporting Cash Payments of Over $10,000 to IRS
- Employer's Tax Guide

#### Persona
Let's assume a persona of a layman who doesn't have an understanding of how IRS works and if some actions have implications or not.

The model will try to answer from the documents in easy language.


## Implementation
In order to follow the RAG approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

- **LLM (Large Language Model)**: Anthropic Claude V1 available through Amazon Bedrock

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the textual documents
- **Document Loader**: PDF Loader available through LangChain

  This is the loader that can load the documents from a source, for the sake of this notebook we are loading the sample files from a local path. This could easily be replaced with a loader to load documents from enterprise internal systems.

- **Vector Store**: FAISS available through LangChain

  In this notebook we are using this in-memory vector-store to store both the embeddings and the documents. In an enterprise context this could be replaced with a persistent store such as AWS OpenSearch, RDS Postgres with pgVector, ChromaDB, Pinecone or Weaviate.
- **Index**: VectorIndex

  The index helps to compare the input embedding and the document embeddings to find relevant document
- **Wrapper**: wraps index, vector store, embeddings model and the LLM to abstract away the logic from the user.

## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In this notebook, we'll also need some extra dependencies:

- [FAISS](https://github.com/facebookresearch/faiss), to store vector embeddings
- [PyPDF](https://pypi.org/project/pypdf/), for handling PDF files

In [2]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet "faiss-cpu>=1.7,<2" langchain==0.0.249 "pypdf>=3.8,<4"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing /root/dependencies/awscli-*-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/root/dependencies/awscli-*-py3-none-any.whl'


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet "faiss-cpu>=1.7,<2" langchain==0.0.309 "pypdf>=3.8,<4"
%pip install --upgrade sqlalchemy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for boto3>=1.28.57 from https://files.pythonhosted.org/packages/d1/71/87b966ec289d84bf65f877e690e715355abf49ed9126af5d44f742dae7be/boto3-1.28.68-py3-none-any.whl.metadata
  Obtaining dependency information for awscli>=1.29.57 from https://files.pythonhosted.org/packages/23/88/b9a29974dc74d233bb3e8d4ee25c7ad15db65edd660316b7a9f18d5f5f52/awscli-1.29.68-py3-none-any.whl.metadata
  Obtaining dependency information for botocore>=1.31.57 from https://files.pythonhosted.org/packages/02/c8/bc962b324537235ed8360520daa862fb77ccb2e64ff08b82af90c3685fb9/botocore-1.31.68-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8874a96c5061422625ba86c3ee1e6587ccd92ff9f5bf7bd91b2/s3transfer-0.7.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 114.8 MB/s eta 0:00:00
 

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [6]:
import json
import os
import sys

import boto3

module_path = "."
sys.path.append(os.path.abspath(module_path))

print(os.path.abspath(module_path))
from utils import bedrock, print_ww

/root/amazon-bedrock-workshop


## Configure langchain

We begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude for text generation and Amazon Titan for text embedding.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

Available model IDs include:

- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`

In [7]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':2000, 'temperature':0})
bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)

In [8]:
import traceback
import time
from typing import Any, Dict, List, Optional
class BedrockEmbeddingsCustom(BedrockEmbeddings):
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Compute doc embeddings using a Bedrock model.

        Args:
            texts: The list of texts to embed

        Returns:
            List of embeddings, one for each text.
        """
        print(f"BedrockEmbeddingsCustom: embed_docs():: lenght of texts={len(texts)}::")
        results = []
        counter = 1
        errors = []
        for text in texts:
            try:
                response = self._embedding_func(text)
                results.append(response)
                #print(f"BedrockEmbeddingsCustom: embed_docs()::processed doc_{counter}:")
                counter+=1
            except:
                print(f"BedrockEmbeddingsCustom: ERROR ={traceback.format_exc()}:: WAITING for 20 SEC")
                time.sleep(20) # 20 sec
                errors.append(text)
        
        print(f"BedrockEmbeddingsCustom: embed_docs(): TRYING Errors now:len={len(errors)}:")
        for text in errors:
            print(f"BedrockEmbeddingsCustom: embed_docs(): error :text={text}:")
            try:
                response = self._embedding_func(text)
                results.append(response)
                #print(f"BedrockEmbeddingsCustom: embed_docs()::processed doc_{counter}:")
                counter+=1
            except:
                print(f"BedrockEmbeddingsCustom: ERROR ={text}:: WAITING for 20 SEC")
                time.sleep(20) # 20 sec
                    
        return results
    
bedrock_embeddings = BedrockEmbeddingsCustom(client=boto3_bedrock)
bedrock_embeddings

BedrockEmbeddingsCustom(client=<botocore.client.BedrockRuntime object at 0x7f9270306e90>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None)

## Data Preparation
Let's first download some of the files to build our document store. For this example we will be using public IRS documents from [here](https://www.irs.gov/publications).

After downloading we can load the documents with the help of [DirectoryLoader from PyPDF available under LangChain](https://python.langchain.com/en/latest/reference/modules/document_loaders.html) and splitting them into smaller chunks.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 512 tokens, which roughly translates to ~2000 characters. For the sake of this use-case we are creating chunks of roughly 1000 characters with an overlap of 100 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

In [22]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFLoader("GenAIAWSnew.pdf")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs = text_splitter.split_documents(documents)

In [23]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 45 documents loaded is 2058 characters.
After the split we have 117 documents more than the original 45.
Average length among 117 documents (after split) is 822 characters.


We had 3 PDF documents which have been split into smaller ~500 chunks.

Now we can see how a sample embedding would look like for one of those chunks

In [24]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 0.23828125 -0.10839844  0.41796875 ... -0.10107422 -0.3828125
 -0.16015625]
Size of the embedding:  (1536,)


Following the similar pattern embeddings could be generated for the entire corpus and stored in a vector store.

This can be easily done using [FAISS](https://github.com/facebookresearch/faiss) implementation inside [LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html) which takes  input the embeddings model and the documents to create the entire vector store. Using the Index Wrapper we can abstract away most of the heavy lifting such as creating the prompt, getting embeddings of the query, sampling the relevant documents and calling the LLM. [VectorStoreIndexWrapper](https://python.langchain.com/en/latest/modules/indexes/getting_started.html#one-line-index-creation) helps us with that.

**⚠️⚠️⚠️ NOTE: it might take few minutes to run the following cell ⚠️⚠️⚠️**

In [25]:
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    bedrock_embeddings,
)

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

vectorstore_faiss.save_local("faiss_index")

BedrockEmbeddingsCustom: embed_docs():: lenght of texts=117::
BedrockEmbeddingsCustom: embed_docs(): TRYING Errors now:len=0:


In [26]:
vectorstore_faiss = FAISS.load_local("faiss_index", bedrock_embeddings)
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

## Question Answering

Now that we have our vector store in place, we can start asking questions.

In [27]:
query = "Invoking Models"

The first step would be to create an embedding of the query such that it could be compared with the documents

In [28]:
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

array([-0.00106049,  0.53515625,  0.7734375 , ...,  0.4609375 ,
        0.20214844, -0.28320312])

We can use this embedding of the query to then fetch relevant documents.
Now our query is represented as embeddings we can do a similarity search of our query against our data store providing us with the most relevant information.

In [29]:
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')
for i, rel_doc in enumerate(relevant_documents):
    print_ww(f'## Document {i+1}: {rel_doc.page_content}.......')
    print('---')

4 documents are fetched which are relevant to the query.
----
## Document 1: Let's assume a persona of a layman who doesn't have an understanding of how IRS works
and if some actions
have implications or not.
The model will try to answer from the documents in easy language.
Implementation
In order to follow the RAG approach this notebook is using the LangChain
framework where it has integrations with different services and tools that allow
efficient building of patterns such as RAG. We will be using the following
tools:
●
●
●
●
●
●LLM (Large Language Model): Anthropic Claude V1 available through Amazon Bedrock
This model will be used to understand the document chunks and provide an answer in human friendly
manner.
Embeddings Model: Amazon Titan Embeddings available through Amazon Bedrock
This model will be used to generate a numerical representation of the textual documents
Document Loader: PDF Loader available through LangChain
This is the loader that can load the documents from a sou

Now we have the relevant documents, it's time to use the LLM to generate an answer based on these documents. 

We will take our inital prompt, together with our relevant documents which were retreived based on the results of our similarity search. We then by combining these create a prompt that we feed back to the model to get our result. At this point our model should give us highly informed information on how we can change the tire of our specific car as it was outlined in our manual.

LangChain provides an abstraction of how this can be done easily.

### Quick way
You have the possibility to use the wrapper provided by LangChain which wraps around the Vector Store and takes input the LLM.
This wrapper performs the following steps behind the scences:
- Takes input the question
- Create question embedding
- Fetch relevant documents
- Stuff the documents and the question into a prompt
- Invoke the model with the prompt and generate the answer in a human readable manner.

Let's ask a different question:

### Customisable option
In the above scenario you explored the quick and easy way to get a context-aware answer to your question. Now let's have a look at a more customizable option with the helpf of [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) where you can customize how the documents fetched should be added to prompt using `chain_type` parameter. Also, if you want to control how many relevant documents should be retrieved then change the `k` parameter in the cell below to see different outputs. In many scenarios you might want to know which were the source documents that the LLM used to generate the answer, you can get those documents in the output using `return_source_documents` which returns the documents that are added to the context of the LLM prompt. `RetrievalQA` also allows you to provide a custom [prompt template](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html) which can be specific to the model.

Note: In this example we are using Anthropic Claude as the LLM under Amazon Bedrock, this particular model performs best if the inputs are provided under `Human:` and the model is requested to generate an output after `Assistant:`. In the cell below you see an example of how to control the prompt such that the LLM stays grounded and doesn't answer outside the context.

In [30]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: You are a machine learning teacher that cares about students. You have to answer the question based on the context. If you don't know the answer, say you don't know the answer. Don't make up an answer if not in context.

Here is the context you should refer to: <context>{context}</context>

Here is the question you should answer based on the context: <question>{question}</question>


Human: Use following instructions to answer the question above.

Assistant: Should I add any attributes in my answer?

Human: Yes. Make sure to include following attributes in your answer as applicable below in <attributes></attributes>
<attributes>
- Give a quick description of the solution in one detailed paragraph 
- Talk about the introduction to how you can solve the machine learning question asked
- Give a code walkthrough and explaination of the code in each line using the <code></code> xml tags
</attributes>

Assistant: Should I add anything else in my answer?

Human: Yes, Provide your output as a text based paragraph where each and every sentence is complete, and grammatically correct. 

Assistant: Can I add anything else in my answer?

Human: Yes divide your answer into the following steps
<steps>
- Have one section in your answer about the introduction to the solution to the problem
- Have one section for the approach to be followed in simple words
- Have one section for the code walkthrough and explaination
- Have one section for how AWS can help the customer use case given in the question
- Have the last section for use cases where the solution can be used in generative AI 
</steps>


Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 9}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
#query = "What are the different ways that the medication lithium can be given?"
query = "How can I invoke a model in AWS Sagemaker?"
result = qa({"query": query})
print_ww(result['result'])

 <answer>
<attributes>
- Give a quick description of the solution in one detailed paragraph
To invoke a model in AWS SageMaker, you first need to create a model artifact by training a machine
learning algorithm on data. Then this model artifact can be deployed to an endpoint, which will host
the model and provide real-time predictions via an API.

- Talk about the introduction to how you can solve the machine learning question asked
AWS SageMaker provides a fully managed service to train, deploy and scale machine learning models.
To leverage SageMaker for a custom model, you first need to containerize your training script and
inference code. Then you can use the SageMaker SDK to kick off a training job, which will output a
model artifact. This can be deployed to a hosted endpoint, allowing you to send requests for real-
time predictions.

- Give a code walkthrough and explaination of the code in each line using the <code></code> xml tags
<code>
# Train a model
model = estimator.fit(inp

### NOW, WHAT MODEL CAN I USE FOR MY USE CASE? (Supports Vision Models, Text Generation, and Code Generation models

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: You are a generative AI model provider that cares about customers. You have to answer the question based on the context. If you don't know the answer, say you don't know the answer. Don't make up an answer if not in context.

Here is the context you should refer to: <context>{context}</context>

Here is the question you should answer based on the context: <question>{question}</question>


Human: Use following instructions to answer the question above.

Assistant: Should I add any attributes in my answer?

Human: Yes. Make sure to include following attributes in your answer as applicable below in <attributes></attributes>
<attributes>
- Give a detailed high level description of the name of the model the user can use
- Give some code related to the model and add some data in your response
- Give examples of where else the model can be used
- Give instances where the model should and should not be used
</attributes>

Assistant: Should I add anything else in my answer?

Human: Yes, Provide your output as a text based paragraph where each and every sentence is complete, and grammatically correct. 

Assistant: Can I add anything else in my answer?

Human: Yes add the following in your answer
<steps>
- Have a code walkthrough of the model name, for example, LLaMa, Cohere, or Falcon that is relevant to the user question
- Make sure to use an example of a model in your answer
- provide data if present in the context about the model
- highlight the importance of sagemaker jumpstart
</steps>


Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 9}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
#query = "What are the different ways that the medication lithium can be given?"
query = "I run a large e-commerce website with over 10,000 products, and I'm looking for a text generation model that can automatically generate product descriptions based on basic information like product name, features, and specifications. I have a dataset of around 50,000 existing product descriptions to train the model. Can you help me find a model that can handle this scale of text generation?"
result = qa({"query": query})
print_ww(result['result'])

#### -------------------------------------------------------------